| [**Overview**](./00_overview.ipynb) |  **Examples:** |  [Selecting and Indexing Geochem Data](01_indexes_selectors.ipynb) | [Data Munging](02_munging.ipynb) | [Visualisation](03_visualisation.ipynb) |[lambdas](04_lambdas.ipynb) |
|:-----|:-----|:-----|:-----|:-----|:-----|

## Geochemical Data - Importing, Processing and 'Munging'

In this notebook we'll go through some of the tasks which `pyrolite` can help make a bit easier with respect to getting your data analysis ready. Here we get to a point we can start playing around with the data in `pyrolite` - by the end the dataset isn't quite cleaned, nor has it gone through any QA - but we can get to some of that after we can access the data in a nice table.

### Importing Data

`pyrolite` is largely based around `pandas`, and as such you're typically working with Pandas dataframes. Pandas can work with [a variety of file types](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html), some more performant than others, but is also happy to digest humble CSV and Excel files (with the functions `pandas.read_csv` and `pandas.read_excel`, respectively). Some of these functions are able to directly connect to remote files (e.g. CSV) or directly to database connections. You'll see one or two examples of fetching a remote CSV file directly below and in other notebooks.

### Cleaning Up Column Names

One of the challenges of working with larger datasets is being able to quickly find the right data when you need it. `pyrolite` provides some functions for this, but for the time being is dependent on being able to recognise compositional columns by looking for elements, oxide names and isotopes (without unit annotations, delimiters and other markup). Here we show some of the steps which might be required to get your dataframe in a standardised format (using examples [from GEOROC](http://georoc.mpch-mainz.gwdg.de/georoc/Entry.html)). Notably, this can be the most difficut step of any analysis workflow, so being able to do this in a way which is repeatable might save you a decent amount of time if you have to do it again in the future!

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Continental_Flood_Basalts_comp/CENTRAL_ATLANTIC_MAGMATIC_PROVINCE_-_CAMP.csv',
                 encoding='cp1252',
                 skip_blank_lines=False) # get some data from GEOROC directly
df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file

Quickly looking at this dataframe, we can see that all the column names are capitalised, `_` is used as a delimiter for isotopes, and units are given for the geochemical parameters in the format `(UNIT)`. We also have a unique index, and ean extra redundant column on the right hand side of our table (which we can drop):

In [2]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,RE187_OS188,HF176_HF177,HE3_HE4,HE3_HE4(R/R(A)),HE4_HE3,HE4_HE3(R/R(A)),K40_AR40,AR40_K40,UNIQUE_ID,Unnamed: 171
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617,NaN
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618,NaN


In [3]:
df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column

We can alter the names of columns which `pyrolite` will recognise so we can use some of its more automated methods. Specifically, pyrolite expects element and oxide names to have 'title case' names (e.g. `Si`, `SiO2`, not `MG`, `MGO`). Similarly, for isotope ratios, it expects something along the lines of `87Sr/86Sr`. Here I've written a function which will attenpt to find relevant element and oxide names among the capitalised versions we find here.

In [4]:
from pyrolite.geochem.ind import __common_elements__, __common_oxides__ # indexes of elements and oxides which we'll check against

def rename_columns(df):
    """
    Rename the columns which pyrolite can access so we can use the indexing and transformation functions.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns you'd like to rename.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe with columns renamed.
    """
    _elements, _oxides = {e.upper(): e for e in __common_elements__}, {o.upper(): o for o in __common_oxides__} # these will serve as lookup tables for our capitalised components
    
    element_columns = {c: _elements[c[:c.find("(PPM")]] for c in df.columns if ("(PPM" in c and c[:c.find("(PPM")]) in _elements} # all of the elemental values are in ppm here
    oxide_columns = {c: _oxides[c[:c.find("(WT%")]] for c in df.columns if ("(WT%" in c and c[:c.find("(WT%")]) in _oxides} # this omits some gas measurements, but gets the major oxides
    isotope_columns = {c: '/'.join([c.title() for c in c.split('_')]) for c in df.pyrochem.list_isotope_ratios if len(c.split('_'))==2}
    
    return df.rename(columns = {**element_columns, **oxide_columns, **isotope_columns})
    

df = rename_columns(df)

In [5]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40,UNIQUE_ID
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618


Finally, it might be a good idea to use the Unique ID as the index for our dataframe:

In [6]:
df = df.set_index('UNIQUE_ID', drop=True)

You could package this up into a function to fetch one of these GEOROC files:

In [7]:
def fetch_GEOROC_csv(filepath):
    """
    Fetch a GEOROC csv from a local file or URL.
    
    Parameters
    ----------
    filepath : str | pathlib.Path
        Filepath to a GEOROC csv - can be used to directly fetch a URL thanks to `pandas.read_csv`.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe formatted for `pyrolite`.
    """
    df = pd.read_csv(filepath, encoding='cp1252', skip_blank_lines=False) # get some data from GEOROC directly
    df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file
    df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column
    df = rename_columns(df)
    df = df.set_index('UNIQUE_ID', drop=True)
    return df

Now we can use this to fetch another CSV - this time from the Kermadec Arc. Note that the column names which `pyrolite` can work with are already converted to usable versions:

In [8]:
kermadec_df = fetch_GEOROC_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Convergent_Margins_comp/KERMADEC_ARC.csv')
kermadec_df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Os187/Os188,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40
UNIQUE_ID,,,,,,,,,,,,,,,,,,,,,
10203-46325,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10203-46329,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
